<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 40px; border-radius: 15px; color: white; text-align: center; margin-bottom: 30px;">
  <h1 style="font-size: 2.5em; margin: 0; font-weight: bold;">📞 Optimización de Horarios de Llamadas</h1>
  <h2 style="font-size: 1.3em; margin-top: 10px; font-weight: 300;">Gestión de Mora Temprana y Tardía</h2>
  <p style="font-size: 1.1em; margin-top: 20px; opacity: 0.9;">Demo Analítica - Proof of Concept</p>
</div>

---

## 📊 Contexto del problema

En la gestión de mora temprana y tardía, una de las principales ineficiencias es la **realización de llamadas en horarios donde los clientes no atienden o evitan el contacto**.

### 🎯 Este notebook demuestra:

- ✅ **Detectar** patrones de atención por cliente
- ✅ **Identificar** franjas horarias más efectivas
- ✅ **Recomendar** el mejor horario para contactar a cada cliente

<div style="background-color: #fff3cd; border-left: 5px solid #ffc107; padding: 15px; margin: 20px 0; border-radius: 5px;">
  <strong>⚡ Objetivo:</strong> Reducir llamadas improductivas y mejorar la tasa de contacto efectivo mediante análisis de datos históricos.
</div>

---

## 🛠️ Stack Técnico

```python
# Librerías utilizadas
import pandas as pd  # Manipulación y análisis de datos
import numpy as np   # Generación de datos sintéticos
```

<div style="display: flex; gap: 10px; margin: 20px 0;">
  <span style="background-color: #e3f2fd; color: #1976d2; padding: 8px 15px; border-radius: 20px; font-weight: bold;">🐍 Python 3.8+</span>
  <span style="background-color: #f3e5f5; color: #7b1fa2; padding: 8px 15px; border-radius: 20px; font-weight: bold;">📊 Pandas</span>
  <span style="background-color: #e8f5e9; color: #388e3c; padding: 8px 15px; border-radius: 20px; font-weight: bold;">🔢 NumPy</span>
</div>

In [ ]:
import pandas as pd
import numpy as np

print("✅ Librerías cargadas correctamente")

---

## 📦 Fase 1: Generación de Datos Sintéticos

Para esta demo, generamos **250 registros históricos** de llamadas:

| Parámetro | Valor |
|-----------|-------|
| **Clientes simulados** | 10 (IDs: 1001-1010) |
| **Intentos por cliente** | 25 |
| **Horario de llamadas** | 08:00 - 21:00 hs |
| **Resultados posibles** | Atendió (35%), No atendió (55%), Rechazó (10%) |

<div style="background-color: #e8f5e9; border-left: 5px solid #4caf50; padding: 15px; margin: 20px 0; border-radius: 5px;">
  <strong>🔒 Privacidad:</strong> Datos 100% sintéticos generados con <code>numpy.random</code>. No se expone información real de clientes.
</div>

In [ ]:
# Configuración de semilla para reproducibilidad
np.random.seed(42)

clientes = range(1001, 1011)  # 10 clientes
registros = []

# Generación de 25 intentos por cliente
for cliente in clientes:
    for _ in range(25):
        hora = np.random.randint(8, 21)
        minuto = np.random.randint(0, 60)
        resultado = np.random.choice(
            ["atendió", "no atendió", "rechazó"],
            p=[0.35, 0.55, 0.10]
        )
        registros.append({
            "cliente_id": cliente,
            "hora": f"{hora:02d}:{minuto:02d}",
            "resultado": resultado
        })

df = pd.DataFrame(registros)

print(f"✅ Dataset generado: {len(df)} registros")
print(f"📊 Primeras filas del dataset:\n")
df.head(10)

---

## ⚙️ Fase 2: Feature Engineering

Transformamos los datos raw en features analíticas:

### 🕐 Segmentación en franjas horarias

| Franja | Rango horario | Características |
|--------|---------------|----------------|
| **Mañana** | 08:00 - 12:00 | Horario laboral temprano |
| **Tarde** | 12:00 - 18:00 | Horario laboral extendido |
| **Noche** | 18:00 - 21:00 | Horario post-laboral |

### ✔️ Variable binaria de atención

Conversión del resultado en variable numérica (`True` = atendió / `False` = no atendió o rechazó)

In [ ]:
# Función para clasificar horarios en franjas
def franja_horaria(hora_str):
    h = int(hora_str.split(":")[0])
    if 8 <= h < 12:
        return "Mañana"
    elif 12 <= h < 18:
        return "Tarde"
    else:
        return "Noche"

# Aplicar segmentación
df["franja"] = df["hora"].apply(franja_horaria)

# Crear variable binaria
df["atendida"] = df["resultado"] == "atendió"

print("✅ Features creadas:")
print(f"   - Franja horaria")
print(f"   - Variable binaria de atención\n")

# Distribución por franja
print("📊 Distribución de intentos por franja:")
print(df["franja"].value_counts().sort_index())
print()

df.head(10)

---

## 📈 Fase 3: Análisis de Patrones

Agregación de datos para identificar **tasas de atención** por cliente y franja horaria.

<div style="background-color: #e3f2fd; border-left: 5px solid #2196f3; padding: 15px; margin: 20px 0; border-radius: 5px;">
  <strong>🔍 Métrica clave:</strong> <code>tasa_atención = atendidas / intentos</code>
  <br><br>
  Esta métrica indica la probabilidad histórica de que un cliente atienda en cada franja horaria.
</div>

In [ ]:
# Análisis agregado por cliente y franja
resumen = (
    df.groupby(["cliente_id", "franja"])
      .agg(
          intentos=("atendida", "count"),
          atendidas=("atendida", "sum")
      )
      .reset_index()
)

# Calcular tasa de atención
resumen["tasa_atencion"] = resumen["atendidas"] / resumen["intentos"]

print("✅ Análisis completado")
print(f"📊 Registros agregados: {len(resumen)}\n")

# Mostrar estadísticas
print("📈 Estadísticas de tasa de atención:")
print(f"   - Promedio general: {resumen['tasa_atencion'].mean():.1%}")
print(f"   - Máximo: {resumen['tasa_atencion'].max():.1%}")
print(f"   - Mínimo: {resumen['tasa_atencion'].min():.1%}\n")

resumen

---

## 🎯 Fase 4: Generación de Recomendaciones

Identificación del **mejor horario** para contactar a cada cliente basado en su tasa histórica de atención.

<div style="background-color: #fff3cd; border-left: 5px solid #ff9800; padding: 15px; margin: 20px 0; border-radius: 5px;">
  <strong>💡 Criterio de selección:</strong> Para cada cliente, se elige la franja horaria con mayor tasa de atención registrada.
</div>

In [ ]:
# Seleccionar mejor franja por cliente
mejor_horario = (
    resumen.sort_values(
        ["cliente_id", "tasa_atencion"],
        ascending=[True, False]
    )
    .groupby("cliente_id")
    .first()
    .reset_index()
)

print("✅ Mejores horarios identificados\n")

# Distribución de recomendaciones
print("📊 Distribución de recomendaciones por franja:")
print(mejor_horario["franja"].value_counts().sort_index())
print()

mejor_horario

---

## 💰 Impacto Empresarial Proyectado

Medición del valor potencial de implementar esta optimización.

In [ ]:
# Calcular métricas de impacto
tasa_actual = df["atendida"].mean()
tasa_optimizada = mejor_horario["tasa_atencion"].mean()
mejora_absoluta = tasa_optimizada - tasa_actual
mejora_relativa = (mejora_absoluta / tasa_actual) * 100

# Llamadas en horarios ineficientes
intentos_totales = len(df)
llamadas_ineficientes = resumen[resumen["tasa_atencion"] < 0.20]["intentos"].sum()
porcentaje_optimizable = (llamadas_ineficientes / intentos_totales) * 100

# Presentación de resultados
print("=" * 70)
print("💰 IMPACTO PROYECTADO DE LA OPTIMIZACIÓN")
print("=" * 70)
print()
print(f"📞 Tasa de contacto actual (baseline):     {tasa_actual:.1%}")
print(f"📈 Tasa proyectada con optimización:       {tasa_optimizada:.1%}")
print(f"🎯 Mejora absoluta:                         +{mejora_absoluta:.1%}")
print(f"📊 Mejora relativa:                         +{mejora_relativa:.1f}%")
print()
print("=" * 70)
print()
print(f"🔍 Llamadas en horarios de baja efectividad (<20%): {llamadas_ineficientes}")
print(f"⚡ Porcentaje de intentos optimizables:              {porcentaje_optimizable:.1f}%")
print()
print("=" * 70)

---

## 📋 Plan de Acción: Recomendaciones Operativas

Conversión del análisis en **instrucciones accionables** para el equipo de gestión de cobranzas.

<div style="background-color: #f3e5f5; border-left: 5px solid #9c27b0; padding: 15px; margin: 20px 0; border-radius: 5px;">
  <strong>🎯 Uso operativo:</strong> Esta lista puede integrarse directamente en sistemas CRM o discadores automáticos para priorizar contactos en los horarios más efectivos.
</div>

In [ ]:
# Generar plan de acción priorizado
print("=" * 70)
print("🎯 PLAN DE LLAMADAS OPTIMIZADO - RECOMENDACIONES INMEDIATAS")
print("=" * 70)
print()

# Top 5 clientes con mejores tasas
top_clientes = mejor_horario.sort_values("tasa_atencion", ascending=False).head(5)

print("📞 TOP 5 CLIENTES CON MAYOR PROBABILIDAD DE CONTACTO:\n")

for idx, row in top_clientes.iterrows():
    cliente = row["cliente_id"]
    franja = row["franja"]
    tasa = row["tasa_atencion"]
    intentos = row["intentos"]
    
    # Definir rangos horarios específicos
    if franja == "Mañana":
        horario = "08:00 - 12:00 hs"
        emoji = "☀️"
    elif franja == "Tarde":
        horario = "12:00 - 18:00 hs"
        emoji = "🌤️"
    else:
        horario = "18:00 - 21:00 hs"
        emoji = "🌙"
    
    print(f"   {emoji} Cliente {cliente}")
    print(f"      → Llamar en: {franja.upper()} ({horario})")
    print(f"      → Probabilidad de atención: {tasa:.0%}")
    print(f"      → Basado en {intentos} intentos históricos")
    print()

print("=" * 70)
print("💡 Recomendación: priorizar estos contactos en las franjas indicadas")
print("   para maximizar la efectividad del equipo de gestión.")
print("=" * 70)

---

## ✅ Conclusiones y Próximos Pasos

### 🎓 Aprendizajes clave

Este notebook demuestra que, a partir de **datos simples de llamadas**, es posible generar **recomendaciones concretas y accionables** para mejorar la eficiencia del contacto telefónico.

### 🚀 Características del enfoque

| Aspecto | Descripción |
|---------|-------------|
| **Escalable** | Puede procesarse con millones de registros en producción |
| **Privado** | Ejecución local sin exponer datos sensibles |
| **Integrable** | Compatible con CRM y sistemas de discado automático |
| **Medible** | Genera métricas claras de impacto empresarial |

### 🔄 Extensiones posibles

- 📅 Análisis por **día de la semana**
- 🎯 Segmentación por **tipo de mora** (temprana/tardía)
- 🗄️ Integración con **bases de datos SQL** (MySQL/PostgreSQL)
- 📊 **Dashboard interactivo** con Plotly/Streamlit
- 🤖 **Automatización** mediante cron jobs
- 🔔 **Alertas** para clientes con comportamiento anómalo

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 15px; color: white; text-align: center; margin-top: 30px;">
  <h3 style="margin: 0;">🎯 Proyecto completado</h3>
  <p style="margin-top: 15px; font-size: 1.1em; opacity: 0.9;">
    De datos históricos a decisiones operativas en 15 celdas
  </p>
</div>